In [1]:
from keras.applications import inception_v3, inception_resnet_v2
from keras.optimizers import SGD, Adam, Nadam
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint,TensorBoard,ReduceLROnPlateau,EarlyStopping,CSVLogger
from keras import backend as K

from sklearn.model_selection import train_test_split
import numpy as np

import os
from tqdm import tqdm
import time
import shutil
import h5py
from PIL import Image

Using TensorFlow backend.


## Training parameters

In [2]:
input_shape = (256, 256, 3)

label_names = {0: 'farm', 1: 'houses', 2: 'others', 3: 'blackfarms'}
label_names_inv = {v: k for k,v in label_names.items()}
num_classes = len(label_names)

batch_size = 16
nb_epoch = 200

np.random.seed(42)

In [3]:
def load_dataset(dataset_path):
    with h5py.File(dataset_path,'r') as f:
        dx = f['data_x'][:]
        dy = f['data_y'][:]  
    return dx, to_categorical(dy, num_classes)

def batch_generator(data_X, data_y, batch_size):
    indexes = np.array(range(len(data_y)))
    n = len(indexes)
    while True:
        batch_start = 0
        batch_end = batch_size
        np.random.shuffle(indexes)
        while batch_start < n:
            index = []
            batch_y = []
            y = []
            index = indexes[batch_start:batch_end]
            batch_x = np.array([data_X[i] for i in index])
            batch_y = np.array([data_y[i] for i in index])
            yield batch_x, batch_y
            batch_start += batch_size   
            batch_end += batch_size

In [4]:
model_name = 'inception_resnet_v2_2d_nadam'
dataset_path = 'dataset_2d_new.h5'
if not os.path.exists(model_name):
    os.mkdir(model_name)

## Load and split data

In [5]:
X, y = load_dataset(dataset_path)
print ('Size of dataset:', len(y))
print(X.shape)

('Size of dataset:', 3975)
(3975, 256, 256, 3)


In [6]:
X_train, y_train = X[1800:], y[1800:]
X_val, y_val = X[600:1200], y[600:1200]
X_test, y_test = X[:600], y[:600]
print('Training: {}\tValidation: {}\tTesting: {}'.format(X_train.shape[0], X_val.shape[0], X_test.shape[0]))

Training: 2175	Validation: 600	Testing: 600


In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)
# print('Training: {}\tValidation: {}\tTesting: {}'.format(X_train.shape[0], X_val.shape[0], X_test.shape[0]))

In [7]:
assert not np.any(np.isnan(X_train))
assert not np.any(np.isnan(y_train))
assert not np.any(np.isnan(X_val))
assert not np.any(np.isnan(y_val))
assert not np.any(np.isnan(X_test))
assert not np.any(np.isnan(y_test))

In [8]:
train_gen = batch_generator(X_train, y_train, batch_size)
val_gen = batch_generator(X_val, y_val, batch_size)
test_gen = batch_generator(X_test, y_test, batch_size)

## Model and training

In [9]:
# model = inception_v3.InceptionV3(include_top=True, weights=None, input_shape=input_shape, classes=num_classes)
model = inception_resnet_v2.InceptionResNetV2(include_top=True, weights=None, input_shape=input_shape, classes=num_classes)
model.load_weights(model_name+'/'+model_name+'.h5')

model.summary()
# sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
# adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=K.epsilon(), decay=0.0)
nadam = Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=K.epsilon(), schedule_decay=0.004)
model.compile(loss='categorical_crossentropy', optimizer=nadam, metrics=['accuracy'])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 127, 127, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 127, 127, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 127, 127, 32) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [14]:
checkpoint = ModelCheckpoint(model_name+'/'+model_name+'.h5', monitor='val_loss', verbose=2, save_best_only=True, mode='auto')
tensorboard = TensorBoard(log_dir=model_name, batch_size=batch_size, write_graph=True, write_images=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001)
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=7, verbose=2, mode='auto')
cvslogger = CSVLogger(model_name+'/'+model_name+'.csv', separator=',', append=True)

callbacks = [checkpoint, reduce_lr, cvslogger, tensorboard, earlystop]
train_steps = int(len(y_train)//batch_size)
val_steps = int(len(y_test)//batch_size)
history = model.fit_generator(train_gen, train_steps, epochs=nb_epoch, verbose=1, 
                    max_queue_size=2, validation_data=val_gen, 
                    validation_steps=val_steps, shuffle=True,
                    workers=1, use_multiprocessing= False, 
                    initial_epoch=0, callbacks=callbacks)

Epoch 1/200
135/135 [==============================] - 32s 239ms/step - loss: 2.2264 - acc: 0.4736 - val_loss: 9.9761 - val_acc: 0.2720

Epoch 00001: val_loss improved from inf to 9.97614, saving model to resnet_50_2d_nadam/resnet_50_2d_nadam.h5
Epoch 2/200
135/135 [==============================] - 27s 197ms/step - loss: 1.8204 - acc: 0.4998 - val_loss: 8.7795 - val_acc: 0.4349

Epoch 00002: val_loss improved from 9.97614 to 8.77954, saving model to resnet_50_2d_nadam/resnet_50_2d_nadam.h5
Epoch 3/200
135/135 [==============================] - 26s 193ms/step - loss: 1.8718 - acc: 0.5089 - val_loss: 3.1691 - val_acc: 0.4161

Epoch 00003: val_loss improved from 8.77954 to 3.16907, saving model to resnet_50_2d_nadam/resnet_50_2d_nadam.h5
Epoch 4/200
135/135 [==============================] - 26s 193ms/step - loss: 1.3204 - acc: 0.5766 - val_loss: 1.0728 - val_acc: 0.5103

Epoch 00004: val_loss improved from 3.16907 to 1.07283, saving model to resnet_50_2d_nadam/resnet_50_2d_nadam.h5
Epoc


Epoch 00039: val_loss did not improve from 0.63542
Epoch 40/200
135/135 [==============================] - 26s 193ms/step - loss: 0.5682 - acc: 0.7934 - val_loss: 0.6895 - val_acc: 0.7774

Epoch 00040: val_loss did not improve from 0.63542
Epoch 41/200
135/135 [==============================] - 26s 193ms/step - loss: 0.5561 - acc: 0.7934 - val_loss: 0.6941 - val_acc: 0.7774

Epoch 00041: val_loss did not improve from 0.63542
Epoch 42/200
135/135 [==============================] - 26s 193ms/step - loss: 0.5817 - acc: 0.7906 - val_loss: 0.6553 - val_acc: 0.7723

Epoch 00042: val_loss did not improve from 0.63542
Epoch 00042: early stopping


## Results and evaluation

In [14]:
model.metrics_names

['loss', 'acc']

In [10]:
import pandas as pd

In [11]:
labelled_full_modified = pd.read_csv('alepores.csv', index_col=0)

print(labelled_full_modified.shape)
labelled_full_modified.head()

(141, 7)


,Correction_from_11_13,Correction_from_13_17,final_label,Long,Lat,x_tile,y_tile
Incorrect,,,,,,,
undefined,construction,construction,construction,36.206474,37.099113,316173,314873
undefined,others,others,others,36.198921,37.147179,316243,314862
undefined,others,others,others,36.161156,37.152672,316251,314807
undefined,others,others,others,36.208534,37.108040,316186,314876
undefined,others,others,others,36.271019,37.145119,316240,314967


In [12]:

def make_filename(x_tile, y_tile, year):
    if year in [11, 13]:
        tempelate = 'gesh_{}_{}_20.jpg'
    else:
        tempelate = 'ges_{}_{}_20.jpg'
    return tempelate.format(x_tile, y_tile)

import cv2

In [13]:
input_shape = (256, 256, 3)

num_examples = labelled_full_modified.shape[0] * 3
test_2x = np.zeros((num_examples,) + input_shape)
test_2y = np.zeros((num_examples, 1))

i = 0
test11='../../Aleppo11/newtask'
test13='../../Aleppo14/newtask'
test17='../../Aleppo18/newtask'

for ind,row in tqdm(labelled_full_modified.iterrows()):
    fname11 = make_filename(row.x_tile, row.y_tile, 11)
    fname13 = make_filename(row.x_tile, row.y_tile, 13)
    fname17 = make_filename(row.x_tile, row.y_tile, 17)
    
    img11 = cv2.imread(os.path.join(test11, fname11))
    img13 = cv2.imread(os.path.join(test13, fname13))
    img17 = cv2.imread(os.path.join(test17, fname17))
    
    if (img11 is None) or (img13 is None) or (img17 is None):
        with open('missing.csv', 'a') as f:
            f.write('missing,missing,{},{},\n'.format(row.x_tile, row.y_tile))
        continue
        
    test_2x[i, :, :, :] = img11
    #data_y[i] = label_names_inv[row.y2011]
    
    test_2x[i+1, :, :, :] = img13
    #data_y[i+1] = label_names_inv[row.y2013]
    
    test_2x[i+2, :, :, :] = img17
    #data_y[i+2] = label_names_inv[row.y2017]
    
    i += 3
print(i)

141it [00:11, 12.24it/s]

423


In [15]:
model.evaluate_generator(test_gen, steps=len(X_test)//batch_size)

[1.0768631028162468, 0.652027027027027]

In [15]:
model.evaluate_generator(test_gen, steps=len(X_test)//batch_size)

[1.0412956540649, 0.6537162162162162]

In [14]:
preds = model.predict(test_2x)
preds_bin = (preds == preds.max(axis=1, keepdims=True)).astype(int)
preds.shape

(423, 4)

In [ ]:
preds[0]

In [ ]:
preds[0]

In [15]:
pres_classes = []
for p in preds.argmax(axis=1):
    pres_classes.append(label_names[p])
    

In [18]:
truth_classes = []
for p in y_test.argmax(axis=1):
    truth_classes.append(label_names[p])

In [16]:
import pandas as pd
df = pd.read_csv('comparison3.csv')
print(df.shape)
df.head()

(0, 5)


,2d_inception_resnetv2,2d_inceptionv3,2d_pred,3d_pred,truth


In [17]:
df['2d_pred'] = pres_classes
df.head()

,2d_inception_resnetv2,2d_inceptionv3,2d_pred,3d_pred,truth
0,NaN,NaN,houses,NaN,NaN
1,NaN,NaN,houses,NaN,NaN
2,NaN,NaN,houses,NaN,NaN
3,NaN,NaN,houses,NaN,NaN
4,NaN,NaN,houses,NaN,NaN


In [21]:
img = Image.fromarray(X_test[2], 'RGB')
img.save('my.png')
img.show()

In [18]:
def transform(val):
    if val.split('_') == ['houses','to','houses']:
        return 'others'
    if val.split('_')[2] == 'houses':
        return 'construction'
    if val == 'blackfarms_to_farm':
        return 'cultivation'
    if val == 'farm_to_blackfarms':
        return 'uncultivation'
    return 'others'

In [19]:
from itertools import product

cc = ['houses', 'farm', 'blackfarms', 'others']
for x in product(cc,cc):
    print(x)

('houses', 'houses')
('houses', 'farm')
('houses', 'blackfarms')
('houses', 'others')
('farm', 'houses')
('farm', 'farm')
('farm', 'blackfarms')
('farm', 'others')
('blackfarms', 'houses')
('blackfarms', 'farm')
('blackfarms', 'blackfarms')
('blackfarms', 'others')
('others', 'houses')
('others', 'farm')
('others', 'blackfarms')
('others', 'others')


In [20]:
labels = []

for index in range(0, len(pres_classes), 3):
    pred11, pred13, pred17 = pres_classes[index:index+3]
    
    trans_11_13 = transform(pred11 + '_to_' + pred13)
    trans_13_17 = transform(pred13 + '_to_' + pred17)
    
    if trans_13_17 != 'others':
        final_label = trans_13_17
    else:
        final_label = trans_11_13
    
    labels.append(final_label)


In [24]:
index

597

In [ ]:
import pandas as pd

In [21]:
df = pd.read_csv('comparison3.csv')
df.head()

,2d_inception_resnetv2,2d_inceptionv3,2d_pred,3d_pred,truth


In [22]:
df['2d_inception_resnetv2'] = labels

In [23]:
df.head()

,2d_inception_resnetv2,2d_inceptionv3,2d_pred,3d_pred,truth
0,others,NaN,NaN,NaN,NaN
1,others,NaN,NaN,NaN,NaN
2,others,NaN,NaN,NaN,NaN
3,others,NaN,NaN,NaN,NaN
4,others,NaN,NaN,NaN,NaN


In [32]:
df.shape()

TypeError: 'tuple' object is not callable

In [24]:
cols = df.columns.tolist()

In [25]:
cols = cols[-1:] + cols[:-1]

In [26]:
cols

['truth', '2d_inception_resnetv2', '2d_inceptionv3', '2d_pred', '3d_pred']

In [27]:
df = df[cols]
df.to_csv('comparison5alepinc.csv', index=False, header=True)
df.head()

,truth,2d_inception_resnetv2,2d_inceptionv3,2d_pred,3d_pred
0,NaN,others,NaN,NaN,NaN
1,NaN,others,NaN,NaN,NaN
2,NaN,others,NaN,NaN,NaN
3,NaN,others,NaN,NaN,NaN
4,NaN,others,NaN,NaN,NaN


In [33]:
df.head()

,truth,2d_inception_resnetv2,2d_inceptionv3,2d_pred,3d_pred
0,NaN,cultivation,NaN,NaN,NaN
1,NaN,others,NaN,NaN,NaN
2,NaN,construction,NaN,NaN,NaN
3,NaN,others,NaN,NaN,NaN
4,NaN,others,NaN,NaN,NaN


### Compare accuracy

In [ ]:
(df['3d_pred'] == df['truth']).sum() / float(len(df))

In [ ]:
(df['2d_pred'] == df['truth']).sum() / float(len(df))

In [ ]:
(df['2d_inceptionv3'] == df['truth']).sum() / float(len(df))

In [ ]:
(df['2d_inception_resnetv2'] == df['truth']).sum() / float(len(df))

In [ ]:
np.sum(np.all(preds_bin == y_test.astype(int), axis=1)) / float(len(y_test))

In [ ]:
import matplotlib.pyplot as plt
import cv2
%matplotlib inline

In [ ]:
class_labels = {0: 'others', 1: 'construction', 2: 'cultivation', 3: 'uncultivation'}

In [ ]:
label_index = 0
for imgnum in tqdm(range(0,len(preds),3)):
#     imgnum = 112

    fig = plt.figure(figsize=(15, 20))
    plt.subplot(131)
    plt.imshow(X_test[imgnum][:,:,::-1])
    plt.axis('off')

    plt.subplot(132)
    plt.imshow(X_test[imgnum+1][:,:,::-1])
    plt.axis('off')

    plt.subplot(133)
    plt.imshow(X_test[imgnum+2][:,:,::-1])
    plt.axis('off')

    pred_label = labels[label_index]
#     truth = class_labels[y_test[imgnum].astype(int)[0]]

    preds_msg = 'pred: {}  /\  truth: X'.format(pred_label)

    fig.text(0.1, 0.6, preds_msg, fontsize=12)

    plt.savefig('final_results_2d/{}.png'.format(imgnum), bbox_inches='tight')
    plt.close()
    
    label_index += 1